#**Pre-request**

##Mount google drive


In [1]:
### **Mount** Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Install pakages


In [2]:
#Install pakages
%pip install -q -r /content/drive/MyDrive/Sem-6/coding/github/fraud_detection/Extract_requirements.txt --no-cache-dir



In [3]:
project_path = "/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/"
%cd $project_path
%ls /content/drive/MyDrive/Sem-6/coding/github/fraud_detection

/content/drive/MyDrive/Sem-6/coding/github/fraud_detection
clustring_requirements-lock.txt  requirements-lock.txt
clustring_requirements.txt       requirements.txt
configs/                         results/
dataset/                         run_experiment.py
Extract_requirements-lock.txt    sample_extract_requirements-lock.txt
Extract_requirements.txt         src/
notebooks/                       tests/
README.md


##Import  libs

In [4]:

import datetime
import os
import pandas as pd
import numpy as np
from scipy.stats import mode
import yaml
import logging
from tqdm import tqdm
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import altair as alt
from google.colab import data_table
data_table.enable_dataframe_formatter()
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta
from datetime import timedelta
import pandas as pd
from tqdm import tqdm
import os
import datetime




%pip freeze > Extract_requirements-lock.txt



#Utility Functions

##Loging

In [5]:

# Make sure results directory exists
os.makedirs("results", exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler("results/data_extract.log")
    ]
)
logger = logging.getLogger(__name__)



##Config

In [6]:
def load_config(config_path="configs/baseline.yaml"):
    """Load YAML config file."""
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    logger.info(f"✅ Loaded config from {config_path}")
    return config


## CDR dataset

In [7]:
def load_cdr(file_path, nrows=None):
    """Load a CSV file and safely parse datetime columns."""
    logger.info(f"📂 Loading file: {file_path} (nrows={nrows})")
    df = pd.read_csv(file_path, nrows=nrows)

    # Auto-detect and parse datetime columns
    for col in df.columns:
        if "datetime" in col.lower() or "time" in col.lower():
            df[col] = pd.to_datetime(df[col], errors="coerce")

    df.columns = df.columns.str.strip()  # clean header spaces
    logger.info(f"✅ Loaded {df.shape[0]} rows, {df.shape[1]} columns")
    return df


def load_all_data(config):
    """
    Load all CSVs defined in config['Agg'] into a dict of DataFrames.
    """
    agg_cfg = config["Agg"]
    base = agg_cfg["base_path"]
    files = agg_cfg["files"]

    data = {}
    for name, fname in files.items():
        path = os.path.join(base, fname)
        df = load_cdr(path)
        data[name] = df
        logger.info(f"📊 Loaded {name:<5} -> {df.shape} from {path}")
    return data

###safe numeric

In [8]:
def safe_numeric(series):
    return pd.to_numeric(series, errors="coerce").fillna(0)


##Month_id_to_end_date

In [9]:
def month_id_to_end_date(month_id):
    """Convert month_id (e.g., 201908, '2019-08', '2019/08') to a month-end datetime safely."""
    if pd.isna(month_id):
        return pd.NaT

    # Convert to string and clean non-digits
    month_str = str(month_id).strip()

    # Handle formats like '2019-08' or '2019/08'
    month_str = month_str.replace("-", "").replace("/", "")

    # If still not purely digits, try generic parse
    if not month_str.isdigit():
        try:
            dt = pd.to_datetime(month_str, errors="coerce")
            return dt + pd.offsets.MonthEnd(0) if pd.notna(dt) else pd.NaT
        except Exception:
            return pd.NaT

    # Handle short formats (e.g., 201908)
    try:
        dt = pd.to_datetime(month_str + "01", format="%Y%m%d", errors="coerce")
        return dt + pd.offsets.MonthEnd(0)
    except Exception:
        return pd.NaT


##Features

###Voice Features

In [10]:
def get_voc_feats(df, cutoff_time=None, n_events=None):
    """Extract per-user voice call features within given time window."""
    df = df.copy()
    if df.empty:
        return pd.DataFrame(columns=["phone_no_m"])

    # ✅ Time filtering (moved from build_user_snapshots)
    if cutoff_time is not None:
        df = df[df["start_datetime"] >= cutoff_time]
    if n_events:
        df = df.sort_values("start_datetime").tail(n_events)

    df["call_dur"] = safe_numeric(df["call_dur"])
    df["weekday"] = pd.to_datetime(df["start_datetime"]).dt.weekday
    df["hour"] = pd.to_datetime(df["start_datetime"]).dt.hour

    feats = (
        df.groupby("phone_no_m", as_index=False)
        .agg(
            voc_total_calls=("start_datetime", "count"),
            voc_unique_contacts=("opposite_no_m", "nunique"),
            voc_total_duration=("call_dur", "sum"),
            voc_avg_duration=("call_dur", "mean"),
            voc_max_duration=("call_dur", "max"),
            voc_std_duration=("call_dur", "std"),
            voc_active_days=("weekday", "nunique"),
            voc_active_hours=("hour", "nunique"),
        )
    )
    return feats.fillna(0)


###SMS Features

In [11]:
def get_sms_feats(df, cutoff_time=None, n_events=None):
    """Extract per-user SMS features within given time window."""
    df = df.copy()
    if df.empty:
        return pd.DataFrame(columns=["phone_no_m"])

    # ✅ Time filtering (moved inside)
    if cutoff_time is not None:
        df = df[df["request_datetime"] >= cutoff_time]
    if n_events:
        df = df.sort_values("request_datetime").tail(n_events)

    df["hour"] = pd.to_datetime(df["request_datetime"]).dt.hour

    feats = (
        df.groupby("phone_no_m", as_index=False)
        .agg(
            sms_total_msgs=("request_datetime", "count"),
            sms_unique_contacts=("opposite_no_m", "nunique"),
            sms_active_hours=("hour", "nunique"),
            sms_calltype_ratio=("calltype_id", lambda x: (x == "out").mean()),
        )
    )
    return feats.fillna(0)


###App Features

In [12]:
def get_app_feats(df):
    """Extract per-user application usage features (aggregated monthly)."""
    df = df.copy()

    if "month_id" not in df.columns:
        raise ValueError("❌ APP dataset must contain 'month_id' column.")

    # ✅ Convert YYYYMM to end-of-month date
    df["month_end"] = df["month_id"].apply(month_id_to_end_date)
    df = df.dropna(subset=["month_id"])

    df["flow"] = pd.to_numeric(df["flow"], errors="coerce").fillna(0)

    monthly = (
    df.groupby(["phone_no_m", "month_end"])
    .agg(
        total_flow=("flow", "sum"),
        unique_apps=("busi_name", "nunique"),
    )
    .reset_index()
)


    features = (
        monthly.groupby("phone_no_m")
        .agg(
            app_months_active=("month_end", "nunique"),
            app_total_flow=("total_flow", "sum"),
            app_avg_flow=("total_flow", "mean"),
            app_std_flow=("total_flow", "std"),
            app_unique_apps_mean=("unique_apps", "mean"),
            app_unique_apps_max=("unique_apps", "max"),
        )
        .reset_index()
    )

    features = features.fillna(0)
    return features


###User Features

In [13]:
def get_user_feats(df):
    """
    Extract per-user features from the user dataset (ARPU monthly columns).
    Converts wide monthly ARPU columns into a long format with arpu_month_id.
    """

    df = df.copy()

    # --- Melt ARPU columns ---
    arpu_cols = [c for c in df.columns if c.startswith("arpu_")]
    if not arpu_cols:
        raise ValueError("❌ No ARPU columns found in user dataset.")

    df_long = df.melt(
        id_vars=["phone_no_m", "city_name", "county_name", "idcard_cnt", "label"],
        value_vars=arpu_cols,
        var_name="arpu_month_id",
        value_name="arpu"
    )

    # --- Convert arpu_YYYYMM -> YYYYMM numeric ---
    df_long["arpu_month_id"] = df_long["arpu_month_id"].str.replace("arpu_", "")
    df_long["arpu_month_id"] = pd.to_numeric(df_long["arpu_month_id"], errors="coerce")

    # --- Convert to month-end date ---
    df_long["arpu_month_end"] = df_long["arpu_month_id"].apply(month_id_to_end_date)

    # --- Convert ARPU to numeric ---
    df_long["arpu"] = pd.to_numeric(df_long["arpu"], errors="coerce").fillna(0)

    # --- Aggregate per user ---
    user_feats = (
        df_long.groupby("phone_no_m")
        .agg(
            user_months_active=("arpu_month_end", "nunique"),
            arpu_mean=("arpu", "mean"),
            arpu_std=("arpu", "std"),
            arpu_max=("arpu", "max"),
            idcard_cnt=("idcard_cnt", "max"),
            label=("label", "max"),
        )
        .reset_index()
    )

    user_feats = user_feats.fillna(0)
    return user_feats


###App Risk Level

In [14]:
def compute_app_risk_levels(df, label_col="label", min_users=5):
    """Compute fraud risk ratio per app."""
    if df.empty or label_col not in df.columns:
        return pd.DataFrame(columns=["busi_name", "risk_level", "fraud_ratio"])

    risk_df = (
        df.groupby("busi_name", as_index=False)
        .agg(
            users=("phone_no_m", "nunique"),
            fraud_users=(label_col, "sum"),
        )
    )
    risk_df["fraud_ratio"] = risk_df["fraud_users"] / risk_df["users"]
    risk_df = risk_df[risk_df["users"] >= min_users]

    # Risk level classification
    bins = [0, 0.01, 0.05, 0.15, 0.30, 1.0]
    labels = ["Low", "Medium", "Elevated", "High", "Critical"]
    risk_df["risk_level"] = pd.cut(risk_df["fraud_ratio"], bins=bins, labels=labels)

    return risk_df

###Time-Aware User Snapshots

In [15]:
import os
import datetime
import pandas as pd
from tqdm import tqdm

def build_user_snapshots_global(
    df_voc,
    df_sms,
    df_app_tx,
    df_arpu_tx,
    window_size=30,           # Can be days or number of events
    window_mode="time",       # "time" or "events"
    max_snapshots=None,
    max_users=None,
    save_path=None
):

    """
    Build synchronized per-user snapshots using unified timeline from all sources.
    Each snapshot = user's state up to a given event timestamp.

    Parameters
    ----------
    df_voc : DataFrame
        Voice call records (must include phone_no_m, start_datetime)
    df_sms : DataFrame
        SMS records (must include phone_no_m, request_datetime)
    df_app_tx : DataFrame
        App transactions (converted by prepare_app_arpu_transactions)
    df_arpu_tx : DataFrame
        ARPU transactions (converted by prepare_app_arpu_transactions)
    window_size : int
        Number of days (if window_mode='time') or number of events (if 'events')
    window_mode : str
        "time" → use a rolling time window; "events" → use last N events
        """
    # Define the expected feature columns (schema)
    ALL_FEATURE_COLUMNS = [
        # Voice
        "voc_total_calls", "voc_unique_contacts", "voc_total_duration",
        "voc_avg_duration", "voc_max_duration", "voc_std_duration",
        "voc_active_days", "voc_active_hours",
        # SMS
        "sms_total_msgs", "sms_unique_contacts", "sms_active_hours", "sms_calltype_ratio",
        # App
        "app_months_active", "app_total_flow", "app_avg_flow",
        "app_std_flow", "app_unique_apps_mean", "app_unique_apps_max",
        # User
        "user_months_active", "arpu_mean", "arpu_std", "arpu_max",
        "idcard_cnt", "label"
    ]

    # --- Step 1: Build unified chronological timeline ---
    df_timeline = build_unified_timeline(df_voc, df_sms, df_app_tx, df_arpu_tx)

    users = sorted(df_timeline["phone_no_m"].unique())
    total_available = len(users)
    if max_users:
        users = users[:max_users]
        print(f"⚙️ Limiting run to first {max_users} users out of {total_available} total.")

    snapshots = []

    # --- Step 2: Iterate through each user ---
    for user in tqdm(users, desc="📸 Building unified snapshots"):
        user_tl = df_timeline[df_timeline["phone_no_m"] == user].reset_index(drop=True)

        if user_tl.empty:
            continue

        # --- Step 2.1: Build user-specific event list with minimum + window filter ---
        user_tl["event_time"] = pd.to_datetime(user_tl["event_time"], errors="coerce")
        user_tl = user_tl.dropna(subset=["event_time"])

        # Each user has their own first valid window start
        event_times_sorted = user_tl["event_time"].sort_values().tolist()
        user_min_time = event_times_sorted[0]

        if window_mode == "time":
            # Start after first window_size days
            user_start_cutoff = user_min_time + datetime.timedelta(days=window_size)
            user_events = [t for t in event_times_sorted if t >= user_start_cutoff]

        elif window_mode == "events":
            # Start after first N events, but still include users with fewer events
            if len(event_times_sorted) > window_size:
                user_events = event_times_sorted[window_size:]
            else:
                # process at least the last available event for users with fewer events
                logger.info(f"ℹ️ User {user} has only {len(event_times_sorted)} events (<{window_size}), using last event.")
                user_events = event_times_sorted[-1:]


        else:
            raise ValueError("Invalid window_mode: must be 'time' or 'events'")


        # If no events remain after filtering, skip this user
        if len(user_events) == 0:
            continue

        total_steps = min(len(user_events), max_snapshots) if max_snapshots else len(user_events)


        for step in range(total_steps):
            cutoff_time = user_events[step]
            event_type = user_tl.loc[user_tl["event_time"] == cutoff_time, "source"].iloc[0]
            event_month_start = cutoff_time.replace(day=1)

            # --- Step 3: Select data for this snapshot ---
            if window_mode == "time":
                start_window = cutoff_time - datetime.timedelta(days=window_size)
                voc_u = df_voc[
                    (df_voc["phone_no_m"] == user)
                    & (df_voc["start_datetime"] > start_window)
                    & (df_voc["start_datetime"] <= cutoff_time)
                ]
                sms_u = df_sms[
                    (df_sms["phone_no_m"] == user)
                    & (df_sms["request_datetime"] > start_window)
                    & (df_sms["request_datetime"] <= cutoff_time)
                ]

            elif window_mode == "events":
                voc_user = df_voc[df_voc["phone_no_m"] == user].sort_values("start_datetime")
                sms_user = df_sms[df_sms["phone_no_m"] == user].sort_values("request_datetime")

                voc_u = voc_user[voc_user["start_datetime"] <= cutoff_time].tail(window_size)
                sms_u = sms_user[sms_user["request_datetime"] <= cutoff_time].tail(window_size)

            else:
                raise ValueError("Invalid window_mode: must be 'time' or 'events'")

            # --- Step 4: App and ARPU features ---
            app_subset = df_app_tx[
                (df_app_tx["phone_no_m"] == user)
                & (df_app_tx["event_time"] < event_month_start)
            ]
            app_feat = get_app_feats(app_subset)

            user_subset = df_arpu_tx[
                (df_arpu_tx["phone_no_m"] == user)
                & (df_arpu_tx["event_time"] < event_month_start)
            ]
            user_feat = get_user_feats(user_subset)

            # --- Step 5: Compute features ---
            voc_feat = get_voc_feats(voc_u)
            sms_feat = get_sms_feats(sms_u)

            # --- Step 6: Ensure all feature frames have phone_no_m ---
            for feat_df in [voc_feat, sms_feat, app_feat, user_feat]:
                if "phone_no_m" not in feat_df.columns:
                    feat_df["phone_no_m"] = [user]

            # --- Step 7: Merge everything into one snapshot ---
            snapshot = (
                pd.DataFrame({
                    "phone_no_m": [user],
                    "snapshot_index": [step + 1],
                    "snapshot_time": [cutoff_time],
                    "event_type": [event_type],
                    "window_mode": [window_mode],
                    "window_size": [window_size],
                })
                .merge(voc_feat, on="phone_no_m", how="left")
                .merge(sms_feat, on="phone_no_m", how="left")
                .merge(app_feat, on="phone_no_m", how="left")
                .merge(user_feat, on="phone_no_m", how="left")
            )
            # ✅ Enforce consistent feature schema and column order
            snapshot = snapshot.reindex(columns=[
                "phone_no_m", "snapshot_index", "snapshot_time",
                "event_type", "window_mode", "window_size"
            ] + ALL_FEATURE_COLUMNS, fill_value=0)

            snapshots.append(snapshot)

    # --- Step 8: Combine results ---
    snapshots_df = pd.concat(snapshots, ignore_index=True).fillna(0)

    # --- Step 9: Save if needed ---
    if save_path:
        if os.path.isdir(save_path):
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            save_file = os.path.join(save_path, f"user_snapshots_{timestamp}.csv")
        else:
            save_file = save_path
        os.makedirs(os.path.dirname(save_file), exist_ok=True)
        snapshots_df.to_csv(save_file, index=False)
        print(f"✅ Snapshots saved to {save_file}")

    print(f"✅ Built {len(snapshots_df)} snapshots across {len(users)} users.")
    return snapshots_df


###Pre proccess

In [16]:
def prepare_app_arpu_transactions(df_app, df_user):
    """
    Convert wide-format APP and USER (ARPU) data into transactional (time-stamped) format.
    Each record will have a proper 'event_time' and 'source' column, aligned for unified timeline merging.
    """

    # --- 1️⃣ APP: Convert month_id or month_str to end-of-month timestamps ---
    df_app_tx = df_app.copy()
    if "month_id" in df_app_tx.columns:
        df_app_tx["event_time"] = pd.to_datetime(df_app_tx["month_id"], errors="coerce") + pd.offsets.MonthEnd(0)
    elif "month_str" in df_app_tx.columns:
        df_app_tx["event_time"] = pd.to_datetime(df_app_tx["month_str"], format="%Y-%m", errors="coerce") + pd.offsets.MonthEnd(0)
    else:
        raise ValueError("df_app must contain either 'month_id' or 'month_str' column.")

    df_app_tx["source"] = "APP"

    # --- 2️⃣ USER (ARPU): Melt monthly ARPU columns into time-stamped transactions ---
    arpu_cols = [c for c in df_user.columns if c.startswith("arpu_")]
    if not arpu_cols:
        raise ValueError("df_user must contain ARPU columns like 'arpu_201908', 'arpu_201909', etc.")

    df_arpu_tx = df_user.melt(
        id_vars=["phone_no_m"],
        value_vars=arpu_cols,
        var_name="month_id",
        value_name="arpu_value"
    )

    # Extract the YYYYMM from the column name (arpu_201908 → 201908)
    df_arpu_tx["month_id"] = df_arpu_tx["month_id"].str.replace("arpu_", "")
    df_arpu_tx["event_time"] = pd.to_datetime(df_arpu_tx["month_id"] + "01", format="%Y%m%d", errors="coerce") + pd.offsets.MonthEnd(0)
    df_arpu_tx["source"] = "ARPU"

    # Attach other static columns (e.g., city_name, county_name, idcard_cnt, label)
    static_cols = [c for c in df_user.columns if not c.startswith("arpu_") and c not in ["phone_no_m"]]
    df_arpu_tx = df_arpu_tx.merge(df_user[["phone_no_m"] + static_cols], on="phone_no_m", how="left")

    print(f"✅ Converted APP to {len(df_app_tx)} transactions and ARPU to {len(df_arpu_tx)} transactions.")
    return df_app_tx, df_arpu_tx


###build_unified_timeline

In [17]:
def build_unified_timeline(df_voc, df_sms, df_app_tx, df_arpu_tx):
    """
    Build a unified chronological timeline of all user-related events (VOC, SMS, APP, ARPU).
    Each entry = (phone_no_m, event_time, source)
    """
    # Standardize time columns
    voc_tl = df_voc[["phone_no_m", "start_datetime"]].rename(columns={"start_datetime": "event_time"})
    voc_tl["source"] = "VOC"

    sms_tl = df_sms[["phone_no_m", "request_datetime"]].rename(columns={"request_datetime": "event_time"})
    sms_tl["source"] = "SMS"

    app_tl = df_app_tx[["phone_no_m", "event_time"]].copy()
    app_tl["source"] = "APP"

    arpu_tl = df_arpu_tx[["phone_no_m", "event_time"]].copy()
    arpu_tl["source"] = "ARPU"

    # Combine all sources
    all_events = pd.concat([voc_tl, sms_tl, app_tl, arpu_tl], ignore_index=True)

    # Drop missing timestamps
    all_events = all_events.dropna(subset=["event_time"])
    all_events["event_time"] = pd.to_datetime(all_events["event_time"], errors="coerce")

    # Sort globally by user and timestamp
    all_events = all_events.sort_values(["phone_no_m", "event_time"]).reset_index(drop=True)

    print(f"📦 Unified timeline built with {len(all_events):,} total events across {all_events['phone_no_m'].nunique()} users.")
    return all_events


#Run Extract

In [18]:
# 1️⃣ Load config and all datasets
config = load_config("/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/configs/baseline.yaml")
data = load_all_data(config)

# 2️⃣ Extract individual datasets from the returned dictionary
df_voc = data["voc"]
df_sms = data["sms"]
df_app = data["app"]
df_user = data["user"]

# 3️⃣ Convert APP and USER (ARPU) data into transaction format
df_app_tx, df_arpu_tx = prepare_app_arpu_transactions(df_app, df_user)
df_timeline = build_unified_timeline(df_voc, df_sms, df_app_tx, df_arpu_tx)

# (Optional) inspect one user:
some_user = df_timeline["phone_no_m"].iloc[0]
display(df_timeline[df_timeline["phone_no_m"] == some_user].head(15))


# 4️⃣ Run unified snapshot builder
save_path = config["Agg"]["save_path"]  # Optional: defined in your config
snapshots_df = build_user_snapshots_global(
    df_voc=df_voc,
    df_sms=df_sms,
    df_app_tx=df_app_tx,
    df_arpu_tx=df_arpu_tx,
    window_size=1000,
    window_mode="events",
    max_users=1,
    max_snapshots=1
)



timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

os.makedirs(save_path, exist_ok=True)
output_file = os.path.join(save_path, f"user_snapshots_{timestamp}.csv")
snapshots_df.to_csv(output_file, index=False)
logger.info(f"✅ Feature extraction completed and saved to: {output_file}")

✅ Converted APP to 26142 transactions and ARPU to 480 transactions.
📦 Unified timeline built with 139,037 total events across 60 users.


,phone_no_m,event_time,source
0,0460a1f64869b7263e73c6ded8390d3afdd9b5eda757bc...,2019-08-31 00:00:00,ARPU
1,0460a1f64869b7263e73c6ded8390d3afdd9b5eda757bc...,2019-09-30 00:00:00,ARPU
2,0460a1f64869b7263e73c6ded8390d3afdd9b5eda757bc...,2019-10-31 00:00:00,ARPU
3,0460a1f64869b7263e73c6ded8390d3afdd9b5eda757bc...,2019-11-30 00:00:00,ARPU
4,0460a1f64869b7263e73c6ded8390d3afdd9b5eda757bc...,2019-12-17 11:28:17,SMS
5,0460a1f64869b7263e73c6ded8390d3afdd9b5eda757bc...,2019-12-17 11:28:17,SMS
6,0460a1f64869b7263e73c6ded8390d3afdd9b5eda757bc...,2019-12-17 11:30:11,SMS
7,0460a1f64869b7263e73c6ded8390d3afdd9b5eda757bc...,2019-12-17 11:30:11,SMS
8,0460a1f64869b7263e73c6ded8390d3afdd9b5eda757bc...,2019-12-17 11:30:11,SMS
9,0460a1f64869b7263e73c6ded8390d3afdd9b5eda757bc...,2019-12-17 11:30:11,SMS


📦 Unified timeline built with 139,037 total events across 60 users.
⚙️ Limiting run to first 1 users out of 60 total.


📸 Building unified snapshots: 100%|██████████| 1/1 [00:00<00:00,  6.49it/s]

✅ Built 1 snapshots across 1 users.
